In [250]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.optimize import minimize
import os

In [251]:
#analysis functions
def getData(filename):
    # read in the data
    df = pd.read_csv(filename)
    # convert to numpy array
    data = df.to_numpy()
    time = data[:, 0]
    gyroZ = data[:, 1]
    in2 = data[:, 2]
    in1 = data[:, 3]
    # cutoff overalocated space where all values are 0
    cuttOffIdx = np.where(time == 0)
    time = time[:cuttOffIdx[0][0]]
    gyroZ = gyroZ[:cuttOffIdx[0][0]]
    in2 = in2[:cuttOffIdx[0][0]]
    in1 = (in1[:cuttOffIdx[0][0]])
    return time, gyroZ, in1, in2

def alignSignals(cutoffTime, time, in1, in2):
    in1Gain = 2.75
    staticTime = np.where(time > cutoffTime)[0][0]
    S1 = in1[:staticTime]
    S2 = in2[:staticTime]
    avg = np.mean(S2)
    def calcGain(offset):
        return np.sum((S1*in1Gain + S2 + offset - 2*avg)**2) + np.sum((S1*in1Gain - S2 + offset)**2)
    # limit offset to 2
    res = minimize(calcGain, [0], bounds=[(-10, 10)])
    offset = res.x[0]
    in1Out = in1*in1Gain + offset
    in2Out = in2
    minValue = np.min([np.min(in1Out), np.min(in2Out)])
    in1Out = in1Out - minValue
    in2Out = in2Out - minValue
    return in1Out, in2Out

def filterSignals(time, in1, in2):
    # filter signals
    fs = 1/np.mean(time[1:]-time[:-1])
    b, a = signal.butter(3, 1, 'low', fs=fs)
    in1 = signal.filtfilt(b, a, in1)
    in2 = signal.filtfilt(b, a, in2)
    return in1, in2

def sumDiff(in1, in2):
    # sum and difference of signals
    Sum = in1+in2
    Diff = in1-in2
    return Sum, Diff

def calcOmega(Sum, Diff, A, Lambda):
    c = 2.99792458e8
    deltaPhi = np.arcsin(Diff/Sum)
    omega = deltaPhi/(8*A*np.pi)*Lambda*c
    return omega

def optimizeTotalPower(gyro, Sum, Diff, A, Lambda):
    # optimize total power so omega's line up, must be in a region where Sum is constant
    def calcErrorOmega(powerOffset):
        omega = calcOmega(Sum+powerOffset, Diff, A, Lambda)
        return np.sum((gyro - omega)**2)
    res = minimize(calcErrorOmega, [0], bounds=[(-10, 10)])
    powerOffset = res.x[0]
    return powerOffset
    
def calcPhi(Sum, Diff):
    deltaPhi = np.arcsin(Diff/Sum)
    return deltaPhi

def calcExpectedPhi(omega, A, Lambda):
    c = 2.99792458e8
    deltaPhi = omega*(8*A*np.pi)/Lambda/c
    return deltaPhi

In [252]:
#plotting functions
def plotSignals(time, gyroZ, in1, in2):
    #plot voltages and angular velocities on same plot with different y axes
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Signal Voltage (V)', color=color)
    ax1.plot(time, in1, 'bo', label='in1', markersize=1)
    ax1.plot(time, in2, 'ro',label='in2', markersize=1)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.legend(loc='upper right')
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('$\Omega$ (rad/s)', color=color)  # we already handled the x-label with ax1
    ax2.plot(time, gyroZ, 'k-',label='gyroZ')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend(loc='lower right')
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    return fig

def plotMathSignals(time, in1, in2, Sum, Diff):
    fig, ax = plt.subplots()
    ax.plot(time, in1, 'bo', label='in1', markersize=1)
    ax.plot(time, in2, 'ro',label='in2', markersize=1)
    ax.plot(time, Sum, 'go',label='in1+in2', markersize=1)
    ax.plot(time, Diff, 'ko',label='in1-in2', markersize=1)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Signal Voltage (V)')
    ax.legend(loc='upper right')
    return fig

def plotFFT(time, in1, in2):
    #plot fft of in1 and in2 on 2 separate plots
    fig, (ax1, ax2) = plt.subplots(1, 2)
    #in1
    # get the fft of the signals
    N = len(in1)
    T = np.mean(time[1:]-time[:-1])
    yf1 = fft(in1)
    yf2 = fft(in2)
    xf = fftfreq(N, T)[:N//2]
    ax1.plot(xf, 2.0/N * np.abs(yf1[0:N//2]), 'bo', label='in1', markersize=1)
    ax2.plot(xf, 2.0/N * np.abs(yf2[0:N//2]), 'ro', label='in2', markersize=1)
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Amplitude')
    ax2.set_xlabel('Frequency (Hz)')
    return fig

def plotOmega(time, gyro, rawOmega, filteredOmega):
    fig, ax = plt.subplots()
    ax.plot(time, rawOmega, 'bo', label='Unfiltered Laser Gyroscope', markersize=1)
    ax.plot(time, filteredOmega, 'ro', label='Filtered Laser Gyroscope', markersize=1)
    ax.plot(time, gyro, 'k-', label='gyroZ')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('$\Omega$ (rad/s)')
    ax.legend(loc='upper right')
    return fig

def plotPhi(time, deltaPhi, expectedDeltaPhi):
    fig, ax = plt.subplots()
    ax.plot(time, deltaPhi, 'bo', label='Measured $\Delta\phi$', markersize=1)
    ax.plot(time, expectedDeltaPhi, 'ro', label='Expected $\Delta\phi$', markersize=1)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('$\Delta\phi$ (rad)')
    ax.legend(loc='upper right')
    return fig


In [253]:
def plotAllAndSave(fname, staticEnd, goodDataStart, goodDataEnd):
    #constants
    A = 0.634*0.438
    Lambda = 632.8e-9
    #strip the file extension and the folder name
    newFolderName = fname.split('/')[-1].split('.')[0]
    #make a new folder to save the plots if it doesn't already exist
    if not os.path.exists(newFolderName):
        os.mkdir(newFolderName)
    time, gyroZ, in1, in2 = getData(fname)
    goodDataStart = np.where(time>goodDataStart)[0][0]
    if (goodDataEnd != -1):
        goodDataEnd = np.where(time>goodDataEnd)[0][0]



    fig = plotSignals(time, gyroZ, in1, in2)
    fig.savefig(newFolderName+'/rawSignals.png')

    in1, in2 = alignSignals(staticEnd,time, in1, in2)
    fig = plotSignals(time, gyroZ, in1, in2)
    fig.savefig(newFolderName+'/alignedSignals.png')
    """
    #will plot the fft of the signals
    fig = plotFFT(time, in1, in2)
    """
    in1Filt, in2Filt = filterSignals(time, in1, in2)
    fig = plotSignals(time, gyroZ, in1Filt, in2Filt)
    fig.savefig(newFolderName+'/filteredAlignedSignals.png')

    Sum, Diff = sumDiff(in1Filt, in2Filt)
    #must be in a region where Sum is constant, manually set the start and end of the region here
    powerOffset = optimizeTotalPower(gyroZ[goodDataStart:goodDataEnd], Sum[goodDataStart:goodDataEnd], Diff[goodDataStart:goodDataEnd], A, Lambda)
    Sum = Sum + powerOffset
    in1Filt  = in1Filt + powerOffset/2
    in2Filt  = in2Filt + powerOffset/2
    omega = calcOmega(Sum, Diff, A, Lambda)
    fig = plotMathSignals(time, in1Filt, in2Filt, Sum, Diff)
    fig.savefig(newFolderName+'/manipulatedSignals.png')

    SumUnfilt, DiffUnfilt = sumDiff(in1, in2)
    SumUnfilt = SumUnfilt + powerOffset
    in1  = in1 + powerOffset/2
    in2  = in2 + powerOffset/2
    omegaUnfilt = calcOmega(SumUnfilt, DiffUnfilt, A, Lambda)
    fig = plotOmega(time, gyroZ, omegaUnfilt, omega)
    fig.savefig(newFolderName+'/omega.png')

    deltaPhis = calcPhi(Sum, Diff)
    expectedDeltaPhis = calcExpectedPhi(gyroZ, A, Lambda)
    fig = plotPhi(time, deltaPhis, expectedDeltaPhis)
    fig.savefig(newFolderName+'/phi.png')
    return

In [254]:
folder = 'goodData/'
plotAllAndSave(folder+'finalSpin5.csv',5,0,-1)
plt.close('all')